In [1]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

In [2]:
folder_path = r"C:\Users\vedan\OneDrive - Indian Institute of Technology Bombay\BTP\Stanford40\ImageSplits"
train = []
test = []
actions = {}

In [3]:
for filename in os.listdir(folder_path):
    if filename == "train.txt":
        file_path = os.path.join(folder_path, filename)
        with open(file_path,"r") as file:
            for line in file:
                train.append(line.strip())
    elif filename == "test.txt":
        file_path = os.path.join(folder_path,filename)
        with open(file_path,"r") as file:
            for line in file:
                test.append(line.strip())
    elif filename == "actions.txt":
        file_path = os.path.join(folder_path, filename)
        with open(file_path,"r") as file:
            next(file)
            idx = 0
            for line in file:
                actions[line.split()[0]] = idx
                idx+=1

In [4]:
class Dataset(Dataset):
    def __init__(self,file_list, actions, image_dir, transform = None):
        self.file_list = file_list
        self.actions = actions
        self.image_dir = image_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_name = self.file_list[idx]
        class_name = "_".join(file_name.split("_")[:-1])
        label = self.actions[class_name]
        image_path = os.path.join(self.image_dir,file_name)
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label
        

In [5]:
def salt_pepper(image):
    np_image = np.array(image)
    h, w, c = np_image.shape
    s_vs_p = 0.5
    amount = 0.02
    noisy = np.copy(np_image)
    num_salt = int(np.ceil(amount * s_vs_p * np_image.size))
    salt_coords = tuple(np.clip(np.random.randint(0, dim, num_salt),0,dim - 1) for dim in np_image.shape[:2])
    noisy[salt_coords[0], salt_coords[1], :] = 255
    num_pepper = int(np.ceil(amount * (1 - s_vs_p) * np_image.size))
    pepper_coords = tuple(np.clip(np.random.randint(0, dim, num_pepper),0,dim - 1) for dim in np_image.shape[:2])
    noisy[pepper_coords[0], pepper_coords[1], :] = 0
    
    return Image.fromarray(noisy.astype(np.uint8))

In [6]:
augmentations = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.Lambda(salt_pepper)], p=0.5),
    transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 2.0)),
    transforms.ToTensor()
])
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [7]:
train_transform = transforms.Compose([augmentations, normalize])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

In [8]:
image_dir = r"C:\Users\vedan\OneDrive - Indian Institute of Technology Bombay\BTP\Stanford40\JPEGImages"
train_dataset = Dataset(train, actions, image_dir, transform=train_transform)
test_dataset = Dataset(test, actions, image_dir, transform=test_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class ModifiedViT(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedViT, self).__init__()
        
        self.model = models.vit_b_16(weights="IMAGENET1K_V1")  

        in_features = self.model.heads.head.in_features   
        self.model.heads.head = nn.Sequential(
            nn.Linear(in_features, 512),  
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),  
            nn.Linear(512, 128), 
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3), 
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3),  
            nn.Linear(64, num_classes)  
        )
    def forward(self, x):
        return self.model(x)

In [10]:
num_classes = 40 
model = ModifiedViT(num_classes=num_classes)
print(model)
model = model.to("cuda" if torch.cuda.is_available() else "cpu")


ModifiedViT(
  (model): VisionTransformer(
    (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (encoder): Encoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (encoder_layer_0): EncoderBlock(
          (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attention): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
            (3): Linear(in_features=3072, out_features=768, bias=True)
            (4): Dropout(p=0.0, inplace=False)
          )
        )
        (encoder_layer_1): EncoderBlock(
          (ln_1): Lay

In [ ]:
def train_and_evaluate(model, train_loader, test_loader, num_epochs, save_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.1, patience=2)
    

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        
   
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

      
        with tqdm(total=len(train_loader), desc="Training", unit="batch") as pbar:
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

     
                outputs = model(images)
                loss = criterion(outputs, labels)

 
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()


                train_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                train_correct += (predicted == labels).sum().item()
                train_total += labels.size(0)

    
                pbar.set_postfix({"Loss": f"{loss.item():.4f}"})
                pbar.update()

        train_accuracy = 100 * train_correct / train_total
        print(f"Train Loss: {train_loss / len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%")

    
        model.eval()
        test_loss = 0.0
        test_correct = 0
        test_total = 0

    
        with tqdm(total=len(test_loader), desc="Testing", unit="batch") as pbar:
            with torch.no_grad():
                for images, labels in test_loader:
                    images, labels = images.to(device), labels.to(device)

                    outputs = model(images)
                    loss = criterion(outputs, labels)

                    test_loss += loss.item()
                    _, predicted = torch.max(outputs, 1)
                    test_correct += (predicted == labels).sum().item()
                    test_total += labels.size(0)

                    pbar.set_postfix({"Loss": f"{loss.item():.4f}"})
                    pbar.update()

        test_accuracy = 100 * test_correct / test_total
        scheduler.step(test_loss / len(test_loader))
        print(f"Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")

    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")



In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device


device(type='cuda')

In [ ]:
save_path = "VIT_weights.pth"
train_and_evaluate(model, train_loader, test_loader, num_epochs=100, save_path=save_path)

Epoch 1/100


Training: 100%|██████████| 125/125 [02:06<00:00,  1.01s/batch, Loss=3.7071]


Train Loss: 3.8388, Train Accuracy: 2.95%


Testing: 100%|██████████| 173/173 [01:02<00:00,  2.77batch/s, Loss=3.2358]


Test Loss: 3.7050, Test Accuracy: 4.75%
Epoch 2/100


...

Test Loss: 0.7662, Test Accuracy: 78.56%
Epoch 100/100


Training: 100%|██████████| 125/125 [02:03<00:00,  1.01batch/s, Loss=0.5465]


Train Loss: 0.4570, Train Accuracy: 89.17%


Testing: 100%|██████████| 173/173 [00:57<00:00,  3.03batch/s, Loss=1.0758]


Test Loss: 0.7572, Test Accuracy: 78.65%
Model saved to mobilenet_v2_weights.pth
